<a href="https://colab.research.google.com/github/AugustinCablant/Viager/blob/main/Bases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Code Stage Augustin Cablant (2023)**

> Ce document regroupe les lignes de code utilisées pour mon stage.

## Imports

In [1]:
!pip install reportlab
!pip install pdfkit
!pip install tqdm
!pip install dateparser
!pip install unidecode
!pip install Levenshtein

import pandas as pd
import numpy as np
from datetime import datetime
import locale
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table
import pdfkit
import tabulate
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import html
import dateparser
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Stage')
from tqdm import tqdm
from unidecode import unidecode
import Levenshtein
from nltk.metrics.distance import jaccard_distance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 12.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 12.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.9 MB/s eta 0:00:00
Mounted at /content/drive


# Phase 1 : création de la base de travail

## DATA FRAME INSEE

> Ce Data Frame regroupe les fichiers de l'INSEE recensant les morts de 2000 à 2022.

> Ce Data Frame regroupe les fichiers de l'INSEE recensant les morts de 1984 à 1999.

> Il suffit de modifier la boucle for ...

In [ ]:
dossier = '/content/drive/MyDrive/Stage/sas/sas/'
# Liste pour stocker les DataFrames
dataframes = []

# Boucle pour créer ma liste de DataFrames

for i in tqdm(range(2000,2023)):
    # Obtenir la liste des fichiers CSV dans le dossier
    chemins_fichiers_sas = dossier+f'a{i}.sas7bdat'
    dataframes.append(pd.read_sas(chemins_fichiers_sas))

df_ins = pd.concat(dataframes)
n = df_ins.shape[0]
df_ins['index']= list(range(1,n+1))
df_ins.set_index('index', inplace=True)

# Data Frame Insee avec les bons noms de colonne et les bons formats
df_insee1 = pd.DataFrame(index=list(range(1,df_ins.shape[0])), columns = ['B_nom', 'B_prenoms',
        'B_sexe', 'B_annee', 'B_mois', 'B_jour','B_ville', 'B_pays', 'B_departement',
        'D_annee', 'D_mois', 'D_jour', 'codebirth', 'nbdeath', 'type', 'reserve', 'fichier', 'allobs',
        'nom','prenoms','sexe','dpt_naissance','ybirth','mbirth','dbirth'])


df_insee1['nom'] = df_ins['nom'].str.decode('utf-8', 'ignore')[2:-1]
df_insee1['B_nom'] = df_ins['nom'].str.decode('utf-8', 'ignore')[2:-1]
df_insee1['B_prenoms'] = df_ins['prenoms'].str.decode('utf-8', 'ignore')[2:-1]
df_insee1['ybirth'] = df_ins['ybirth']
df_insee1['mbirth'] = df_ins['mbirth']
df_insee1['dbirth'] = df_ins['dbirth']
df_insee1['B_annee'] = df_ins['ybirth']
df_insee1['B_mois'] = df_ins['mbirth']
df_insee1['B_jour'] = df_ins['dbirth']
df_insee1['D_annee'] = df_ins['ydeath']
df_insee1['D_mois'] = df_ins['mdeath']
df_insee1['D_jour'] = df_ins['ddeath']
df_insee1['dpt_naissance'] = (df_ins['codebirth'].str.decode('utf-8')).str.slice(0,2)
df_insee1['B_departement'] = (df_ins['codebirth'].str.decode('utf-8')).str.slice(0,2)
df_insee1['B_pays'] = df_ins['paysbirth'].str.decode('iso-8859-1')
df_insee1['B_ville'] = df_ins['villebirth'].str.decode('iso-8859-1')
df_insee1['codebirth'] = df_ins['codebirth']
df_insee1['codebirth'] = df_ins['codebirth']
df_insee1['nbdeath'] = df_ins['nbdeath']
df_insee1['type'] = df_ins['type']
df_insee1['reserve'] = df_ins['reserve']
df_insee1['fichier'] = df_ins['fichier']
df_insee1['allobs'] = df_ins['allobs']
df_insee1['B_sexe'] = df_ins['sex']
df_insee1['sexe'] = df_ins['sex']
df_insee1['B_departement']= pd.to_numeric(df_insee1['B_departement'], errors='coerce').astype(float)
df_insee1['dpt_naissance']= pd.to_numeric(df_insee1['B_departement'], errors='coerce').astype(float)

100%|██████████| 23/23 [03:16<00:00,  8.55s/it]


## DATA FRAME AGENCE

In [ ]:
df_grille = pd.read_csv('data_agence.csv', encoding='ISO-8859-1')

def convert_to_timestamp(value):
    if pd.isnull(value) or value == '/':
        return value
    else:
        try:
            timestamp = dateparser.parse(str(value))
            return timestamp
        except ValueError:
            return value

df_grille['Date_naissance_vendeur_1'] = df_grille['Date_naissance_vendeur_1'].apply(convert_to_timestamp)
df_grille['Date_naissance_vendeur_2'] = df_grille['Date_naissance_vendeur_2'].apply(convert_to_timestamp)
df_grille = df_grille.dropna(subset = 'Tete_1')
df_grille['Tete_1'] = df_grille['Tete_1'].astype(str)
df_grille['Tete_2'] = df_grille['Tete_2'].astype(str)
df_grille[df_grille['Tete_1']=='Vendeur 1']['lieutete1'] = df_grille[df_grille['Tete_1']=='Vendeur 1']['Ville_naissance_vendeur_1']
df_grille[df_grille['Tete_1']=='Vendeur 2']['lieutete1'] = df_grille[df_grille['Tete_1']=='Vendeur 2']['Ville_naissance_vendeur_2']
df_grille[df_grille['Tete_2']=='Vendeur 2']['lieutete2'] = df_grille[df_grille['Tete_2']=='Vendeur 2']['Ville_naissance_vendeur_2']


colonnes = ['Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'nom1', 'nom2', 'prenom1', 'prenom2', 'sexe1', 'sexe2',
       'annee1', 'annee2', 'naissancetete1', 'mois1', 'jour1',
       'naissancetete2', 'mois2', 'jour2', 'lieutete1', 'dep1', 'lieutete2',
       'dep2', 'abroad1', 'abroad2', 'quality', 'nom', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

colonnes1 = ['nom1', 'prenom1', 'annee1', 'mois1', 'jour1',
       'dep1', 'abroad1','lieutete1','sexe1',
        'Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'quality', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

colonnes2 = ['nom2', 'prenom2', 'annee2', 'mois2', 'jour2',
       'dep2', 'abroad2','lieutete2','sexe2',
       'Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'quality', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

df_travail = pd.DataFrame(index = list(range(1,df_grille['nom1'].count()+df_grille['nom2'].count()+1)),
                          columns = colonnes)

df1 = df_grille[pd.notnull(df_grille['nom1'])][colonnes1]
df2 = df_grille[pd.notnull(df_grille['nom2'])][colonnes2]

df1 = df1.rename(columns={'nom1': 'A_nom', 'prenom1' : 'A_prenoms',
                          'annee1' : 'A_annee' , 'mois1' : 'A_mois',
       'jour1' : 'A_jour', 'lieutete1' : 'A_ville',
                          'dep1' : 'A_departement',
       'abroad1':'A_abroad', 'sexe1' : 'A_sexe'})
df2 = df2.rename(columns={'nom2': 'A_nom', 'prenom2' : 'A_prenoms',
                          'annee2' : 'A_annee' , 'mois2' : 'A_mois',
       'jour2' : 'A_jour', 'lieutete2' : 'A_ville',
                          'dep2' : 'A_departement',
       'abroad2':'A_abroad', 'sexe2' : 'A_sexe'})


df_travail = pd.concat([df1, df2], ignore_index = True)
df_travail['A-departement'] = pd.to_numeric(df_travail['Dpt_naissance'], errors='coerce').astype('Int64').astype(str)
df_travail['A_sexe'] = df_travail['sexe'].astype(float)
df_travail['A_nom'] = df_travail['nom'].str.upper()

# création des colonnes de match avec le df INSEE
df_travail['nom'] = df_travail['A_nom']
df_travail['sexe'] = df_travail['A_sexe']
df_travail['ybirth'] = df_travail['A_annee']
df_travail['mbirth'] = df_travail['A_mois']
df_travail['dbirth'] = df_travail['A_jour']
df_travail['dpt_naissance'] = df_travail['A_departement']

# colonne pour pouvoir identifier les individus (et non plus les paires de têtes)
df_travail['identification'] = list(range(1,df_travail.shape[0]+1))
df_travail = df_travail.append(df_grille[])

## MERGE

In [ ]:
df_travail = pd.read_csv('df_travail.csv')
merged_df = df_travail.merge(df_insee1, on=['nom','sexe','ybirth','mbirth','dbirth','dpt_naissance'], how='left')
merged_df = merged_df.dropna(subset = ['ydeath'] )

# Vérification à l'aide de la liste des prénoms :
merged_df['prenoms_x'] = merged_df['prenoms_x'].str.upper()
merged_df['prenoms_y'] = merged_df['prenoms_y'].str.upper()
liste_verif = []
for i in tqdm(merged_df.index.tolist()):

  liste_unidecode = []
  liste_prenoms = []

  for element in str(merged_df['prenoms_y'][i]).split(' '):
    liste_unidecode.append(unidecode(element).strip())
    liste_prenoms.append(element.strip())

  if len(str(merged_df['prenoms_x'][i]).split(' '))==1 and pd.notnull(merged_df['prenoms_x'][i]) :

    if merged_df['prenoms_x'][i].strip() in liste_prenoms:
      liste_verif.append(merged_df['identification'][i])
    elif unidecode(merged_df['prenoms_x'][i]) in liste_unidecode :
      liste_verif.append(merged_df['identification'][i])
    elif pd.notnull(merged_df['prenoms_y'][i]) and unidecode(merged_df['prenoms_x'][i]).strip() == unidecode(merged_df['prenoms_y'][i]):
      liste_verif.append(merged_df['identification'][i])
    else:
      pass

  elif pd.notnull(merged_df['prenoms_x'][i]) and len(merged_df['prenoms_x'][i].split(' '))>1:
    for element in merged_df['prenoms_x'][i].split(' '):
      if element in liste_prenoms or unidecode(element) in liste_unidecode:
        liste_verif.append(merged_df['identification'][i])
  else:
    pass

liste_verif = list(set(liste_verif))
print("Il y a",len(liste_verif),"individus pour qui les prénoms coïncident")